In [5]:
import pandas as pd

In [6]:
recipes = pd.read_csv('./output_recipes.csv')

new = pd.read_csv('./existence.csv')

user_db = pd.read_csv('./user_db1.csv')

In [7]:

def find_matching_recipes(user_id_input, password_input, ingredients, vegan_vege=False, dish_type=None, healthy=False):


    def login(user_id_input, password_input):
        user_data = user_db[user_db['id'] == user_id_input]
        if not user_data.empty and user_data['password'].values[0] == password_input:
            return user_data
        return None

    def find_top_matching_rows(user_data, matching_rows):
        common_ingredients = [col for col in user_data.columns if col not in ['id', 'password']]
        matching_rows['total_score'] = matching_rows[common_ingredients].mul(user_data[common_ingredients].values[0]).sum(axis=1)
        top_5_rows = matching_rows.nlargest(5, 'total_score')
        return top_5_rows['id'].tolist()

    matching_rows = pd.DataFrame()

    for ingredient in ingredients:
        if ingredient in new.columns:
            matching_rows = new[new[ingredient] == 1]

    if vegan_vege:
        matching_rows = matching_rows[matching_rows['vegan/vege'].isin([True, False])]

    if dish_type:
        matching_rows = matching_rows[matching_rows['food type'].isin(dish_type)]

    if healthy:
        matching_rows = matching_rows[matching_rows['healthiness'].isin([True, False])]

    matching_rows.reset_index(drop=True, inplace=True)

    user_data = login(user_id_input, password_input)

    if user_data is not None:
        user_data = user_data.reset_index(drop=True)
        top_5_matching_ids = find_top_matching_rows(user_data, matching_rows)
        if top_5_matching_ids:
            return top_5_matching_ids
        else:
            return []
    else:
        return None



In [8]:
user_id = 'TEST122'
password = 'TEST122'
ingredients = ['beef']
vegan_vege = False
dish_type = ['dish']
healthy = False

top_5_matching_ids = find_matching_recipes(user_id, password, ingredients, vegan_vege, dish_type, healthy)

if top_5_matching_ids:
    print("Top 5 matching dish IDs:")
    print(top_5_matching_ids)
else:
    print("No matching dishes found.")

Top 5 matching dish IDs:
[461060, 88743, 64535, 434460, 385128]
